## First Layer

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager

import datetime
import time
from bs4 import BeautifulSoup
import urllib.request
import pandas as pd
import numpy as np
from datetime import date
date_today = date.today()

options = webdriver.ChromeOptions()
options.add_argument("start-maximized")
options.add_argument("--disable-extensions")
driver = webdriver.Chrome(options=options, executable_path=r'/Users/ricardolu/Desktop/Design lab project/chromedriver')
driver.set_window_position(0, 0)
driver.set_window_size(1248, 768)

driver.get("https://us.topshop.com/en/tsus/category/clothing-70483?currentPage=1")
time.sleep(1.5)

counter = 1

for i in range(1,30):
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:

        # Scroll down to bottom
        driver.execute_script("window.scrollBy(0, 768);")

        # Wait to load page
        time.sleep(.2)

        # Calculate new scroll height and compare with last scroll height
        new_height = 768 * counter
        if new_height >= last_height:
            time.sleep(5)
            break

        counter += 1

soup = BeautifulSoup(driver.page_source,'html.parser')

In [2]:
product_name = []
original_price = []
sale_price = []
product_url = []
image_url = []
brand = []
product_type = []
timestamp = []


for i in soup.find_all('div',{'class':'Product Product--col3'}):
    if i.find('a',{'class':'Product-nameLink'}) != None:
        product_name.append(i.find('a',{'class':'Product-nameLink'}).text.strip())
    else:
        product_name.append(None)

    if i.find('span',{'class':'HistoricalPrice-old HistoricalPrice-was'}) != None:
        original_price.append((i.find('span',{'class':'Price notranslate'})).text.strip())
        sale_price.append((i.find_all('span',{'class':'Price notranslate'})[1]).text.strip())
    elif i.find('span',{'class':'Price notranslate'}) != None:
        sale_price.append((i.find('span',{'class':'Price notranslate'})).text.strip())
        original_price.append(None)
    else:
        sale_price.append(None)
        original_price.append(None)
        

    if i.find('a',{'class':'Product-link'}) != None:
        product_url.append("https://us.topshop.com"+i.find('a',{'class':'Product-link'}).get('href'))
    else:
        product_url.append(None)

    if i.find('img',{'class':'ProductImages-image ProductImages-mobile--showing'}) !=None:
        image_url.append("https:"+(i.find('img',{'class':'ProductImages-image ProductImages-mobile--showing'})).get('src'))
    else:
        image_url.append(None)
    brand.append("topshop")
    product_type.append("Women's Clothing")
    timestamp.append(date_today)

In [3]:
column_names = ["product_name", "original_price", "sale_price",'product_link', 'image_link']
df = pd.DataFrame(columns = column_names)

df['product_name'] = product_name
df['original_price'] = original_price
df['sale_price'] = sale_price
df['image_link'] = image_url
df['product_link'] = product_url

In [4]:
df

,product_name,original_price,sale_price,product_link,image_link
0,Black Jamie Skinny Jeans,None,$70.00,https://us.topshop.com/en/tsus/product/clothin...,https://images.topshop.com/i/TopShop/TS02Y17IB...
1,Camel Classic Double Breasted Coat,None,$135.00,https://us.topshop.com/en/tsus/product/clothin...,https://images.topshop.com/i/TopShop/TS07C20TC...
2,Mid Blue Jamie Skinny Jeans,None,$70.00,https://us.topshop.com/en/tsus/product/clothin...,https://images.topshop.com/i/TopShop/TS02K12SM...
3,CONSIDERED Washed Black Straight Jeans,None,$75.00,https://us.topshop.com/en/tsus/product/clothin...,https://images.topshop.com/i/TopShop/TS02G03SW...
4,Bleach Stone Premium Mom Tapered Jeans,None,$90.00,https://us.topshop.com/en/tsus/product/clothin...,https://images.topshop.com/i/TopShop/TS02P21SB...
...,...,...,...,...,...
715,IDOL Charcoal Gray Frill Cable Knitted Sweater...,None,$68.00,https://us.topshop.com/en/tsus/product/clothin...,https://images.topshop.com/i/TopShop/TS23W06TC...
716,Navy Check Shacket and Jeans Co-ord,$165.00,$106.00,https://us.topshop.com/en/tsus/product/clothin...,https://images.topshop.com/i/TopShop/BUNDLE_05...
717,IDOL Spot Ruffle Mini Dress,None,$75.00,https://us.topshop.com/en/tsus/product/clothin...,https://images.topshop.com/i/TopShop/TS10M54TM...
718,AYLA Black Leather Chunky Sock Boots,None,$160.00,https://us.topshop.com/en/tsus/product/shoes-7...,https://images.topshop.com/i/TopShop/TS32A19TB...


## Second Layer

In [13]:
product_description = []
material = []
picture_name = []
color = []
counter = 1
driver = webdriver.Chrome(options=options, executable_path=r'/Users/ricardolu/Desktop/Design lab project/chromedriver')

for link in product_url:
    driver.get(link)
    time.sleep(1.5)

    soup = BeautifulSoup(driver.page_source,'html.parser')
    
    if soup.find('h1',{'class':'ProductDetail-title'}) != None:
        if soup.find('div',{'class':'ProductDescription-content'}) != None:
            product_description.append(soup.find('div',{'class':'ProductDescription-content'}).text.strip())
        else:
            product_description.append(None)

        material_item = []
        if soup.find('div',{'class':'ProductDescription-content'}) != None:
            material.append(soup.find('div',{'class':'ProductDescription-content'}).text.strip())
        else:
            material.append(None)

        if soup.find('li',{'class':'ProductDescriptionExtras-item'}) != None:
            color.append(soup.find('li',{'class':'ProductDescriptionExtras-item'}).text.strip().replace("Colour:",''))
    else:
        color.append(None)
        product_description.append(None)
        material.append(None)

    picture_name.append('topshop-women_' + str(counter) + '.jpg') # change variable name
    counter += 1

In [19]:
column_names = ["brand", "product_type", "product_name", 'product_description', "price", 
                "sales_price", "color","material", "product_url", "picture_url", "timestamp", 'picture_name']
second_pass = pd.DataFrame(columns = column_names)
second_pass.brand = brand
second_pass.product_type = product_type
second_pass.product_name = product_name
second_pass.product_description = product_description
second_pass.price = original_price
second_pass.sales_price = sale_price
second_pass.color = color
second_pass.material = material
second_pass.product_url = product_url
second_pass.picture_url = image_url
second_pass.timestamp = timestamp
second_pass.picture_name = picture_name
second_pass

,brand,product_type,product_name,product_description,price,sales_price,color,material,product_url,picture_url,timestamp,picture_name
0,topshop,Women's Clothing,Black Jamie Skinny Jeans,Introducing our most popular style and most au...,None,$70.00,BLACK,Introducing our most popular style and most au...,https://us.topshop.com/en/tsus/product/clothin...,https://images.topshop.com/i/TopShop/TS02Y17IB...,2020-12-24,topshop-women_1.jpg
1,topshop,Women's Clothing,Camel Classic Double Breasted Coat,This autumn winter step out in timeless style ...,None,$135.00,CAMEL,This autumn winter step out in timeless style ...,https://us.topshop.com/en/tsus/product/clothin...,https://images.topshop.com/i/TopShop/TS07C20TC...,2020-12-24,topshop-women_2.jpg
2,topshop,Women's Clothing,Mid Blue Jamie Skinny Jeans,Our most popular style and most authentic skin...,None,$70.00,MID STONE,Our most popular style and most authentic skin...,https://us.topshop.com/en/tsus/product/clothin...,https://images.topshop.com/i/TopShop/TS02K12SM...,2020-12-24,topshop-women_3.jpg
3,topshop,Women's Clothing,CONSIDERED Washed Black Straight Jeans,Transform your collection with this twist on a...,None,$75.00,Washed Black,Transform your collection with this twist on a...,https://us.topshop.com/en/tsus/product/clothin...,https://images.topshop.com/i/TopShop/TS02G03SW...,2020-12-24,topshop-women_4.jpg
4,topshop,Women's Clothing,Bleach Stone Premium Mom Tapered Jeans,Inspired by our best-selling premium Mom short...,None,$90.00,BLEACH STONE,Inspired by our best-selling premium Mom short...,https://us.topshop.com/en/tsus/product/clothin...,https://images.topshop.com/i/TopShop/TS02P21SB...,2020-12-24,topshop-women_5.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...
715,topshop,Women's Clothing,IDOL Charcoal Gray Frill Cable Knitted Sweater...,Our charcoal gray cable sweater tank is part o...,None,$68.00,CHARCOAL,Our charcoal gray cable sweater tank is part o...,https://us.topshop.com/en/tsus/product/clothin...,https://images.topshop.com/i/TopShop/TS23W06TC...,2020-12-24,topshop-women_716.jpg
716,topshop,Women's Clothing,Navy Check Shacket and Jeans Co-ord,None,$165.00,$106.00,None,None,https://us.topshop.com/en/tsus/product/clothin...,https://images.topshop.com/i/TopShop/BUNDLE_05...,2020-12-24,topshop-women_717.jpg
717,topshop,Women's Clothing,IDOL Spot Ruffle Mini Dress,Channel some new-season pattern with our all o...,None,$75.00,MULTI,Channel some new-season pattern with our all o...,https://us.topshop.com/en/tsus/product/clothin...,https://images.topshop.com/i/TopShop/TS10M54TM...,2020-12-24,topshop-women_718.jpg
718,topshop,Women's Clothing,AYLA Black Leather Chunky Sock Boots,Love your footwear with a fashionable twist wi...,None,$160.00,BLACK,Love your footwear with a fashionable twist wi...,https://us.topshop.com/en/tsus/product/shoes-7...,https://images.topshop.com/i/TopShop/TS32A19TB...,2020-12-24,topshop-women_719.jpg


In [20]:
second_pass.to_csv("topshop-women.csv", index=False, encoding = 'utf-8-sig')

In [9]:
import os
import requests as req

new_pd = pd.read_csv('topshop-women.csv', encoding='utf-8-sig')

os.mkdir('topshop-women')
i = 1
for index, link in enumerate(new_pd['picture_url']):
    headers = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36'}
    img_data = req.get(link,headers=headers).content
    with open('topshop-women/'+ "topshop-women_"+str(index+1)+'.jpg','wb+') as f:
        f.write(img_data)
    i+=1